In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/MALGUNSL.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [ ]:
def df_str(df):
    #전체 리뷰들을 하나의 문자열로 만듦
    s = [s for s in df]
    document = ""
    for i in range(len(s)):
        document += s[i] # 대입연산자 document = document+s[i]와 같은 의미이고 이렇게 코딩해도 됩니다. 
    return document #데이터프레임의 원소들을 잘게 쪼개어 문자열(리스트)로 만들어주는 역할을 합니다. 

def get_noun(document):# document에는 문자열 리스트라면 뭐든지 들어갈 수 있습니다.
    # okt tagger를 이용해 명사 추출
    okt = Okt() # Okt()클래스를 인스턴스 한 것인데요 이건 개발쪽 지식에 가까우니 그냥 넘어가셔도 됩니다.
    noun = okt.nouns(document) #nouns()는 문자열에서 명사인 것들만 추려오는 역할을 하는 함수입니다. 
    for i in enumerate(noun):#enumerate는 데이터의 집합체(컬렉션)로부터 값을 읽어오는 기능을 하는 함수입니다. 
        if len(i)<2:
            noun.pop(i)
    count = Counter(noun)
    noun_list = count.most_common(100)#빈도수가 높은 100개의 단어를 가져오라는 뜻이에요

    return noun_list

def wordcloud(noun_list, title):
    # 워드클라우드 이미지 생성
    wc= WordCloud(
        font_path = 'C:\WINDOWS\FONTS\MALGUNSL.TTF',    # 한글폰트 경로 설정
        background_color='white',                             # 배경 색깔 정하기
        colormap = 'Dark2',                                # 폰트 색깔 정하기
        width = 800,
        height = 800).generate_from_frequencies(dict(noun_list)) 
    
    plt.figure(figsize=(10,10)) #이미지 사이즈 지정
    plt.suptitle("Word Cloud", fontsize=40)
    plt.title(title, fontsize=20)
    plt.imshow(wc, interpolation='lanczos') #이미지의 부드럽기 정도
    plt.axis('off') #x y 축 숫자 제거
    plt.show()  # 워드클라우드 이미지 확인
    
    return wc

In [ ]:
def m_p_wc(dd):
    mask_pos = (dd.label == 1)
    mask_neg = (dd.label == 0)
    df_total = dd.review.copy()
    df_pos = dd.loc[mask_pos,:].review # 긍정 리뷰에만 해당하는 데이터
    df_neg = dd.loc[mask_neg,:].review # 부정 리뷰에만 해당하는 데이터
    noun_list = get_noun(di)
    document_p = df_str(df_pos) # 부정 리뷰 데이터프레임을 하나의 큰 문자열로 만듭니다.
    noun_list_p = get_noun(document_p) # 명사만을 추출하고 빈도수를 계산합니다.
    document_n = df_str(df_neg) # 부정 리뷰 데이터프레임을 하나의 큰 문자열로 만듭니다.
    noun_list_n = get_noun(document_n) # 명사만을 추출하고 빈도수를 계산합니다.
    neg = list(dict(noun_list_n).keys()) # key(명사) 만을 이용해 리스트 생성
    pos = list(dict(noun_list_p).keys())
    drop_words = [w for w in neg if w in pos] # 부정리뷰에 포함된 명사가 긍정리뷰에도 등장할 경우 drop_words라는 리스트에 저장
    noun_list_p_filter = [x for x in noun_list_p if x[0] not in drop_words] # drop_words 리스트에 포함되지 않은 긍정리뷰 명사를 저장
    
    positive_reviews_filtered = wordcloud(noun_list_p_filter, '긍정 리뷰 (필터링 됨)')
        
    return positive_reviews_filtered

def m_n_wc(dd):
    mask_pos = (dd.label == 1)
    mask_neg = (dd.label == 0)
    df_total = dd.review.copy()
    df_pos = dd.loc[mask_pos,:].review # 긍정 리뷰에만 해당하는 데이터
    df_neg = dd.loc[mask_neg,:].review # 부정 리뷰에만 해당하는 데이터
    noun_list = get_noun(di)
    document_p = df_str(df_pos) # 부정 리뷰 데이터프레임을 하나의 큰 문자열로 만듭니다.
    noun_list_p = get_noun(document_p) # 명사만을 추출하고 빈도수를 계산합니다.
    document_n = df_str(df_neg) # 부정 리뷰 데이터프레임을 하나의 큰 문자열로 만듭니다.
    noun_list_n = get_noun(document_n) # 명사만을 추출하고 빈도수를 계산합니다.
    neg = list(dict(noun_list_n).keys()) # key(명사) 만을 이용해 리스트 생성
    pos = list(dict(noun_list_p).keys())
    drop_words = [w for w in neg if w in pos] # 부정리뷰에 포함된 명사가 긍정리뷰에도 등장할 경우 drop_words라는 리스트에 저장
    noun_list_n_filter = [x for x in noun_list_n if x[0] not in drop_words] # drop_words 리스트에 포함되지 않은 부정리뷰 명사를 저장
    
    negative_reviews_filtered = wordcloud(noun_list_n_filter, '부정 리뷰(필터링 됨)')
    
    return negative_reviews_filtered